##Pipeline de limpieza gusanos 

In [2]:
import pandas as pd

# Cargar datos
df = pd.read_csv("gusanos.csv", sep=",")

# Vista general
print("Dimensiones:", df.shape)
display(df.head())

# Tipos de datos
df.info()


Dimensiones: (177, 108)


,Nombre,Apellido,Rut,Foto cédula,Edad,Telefono,Ubicacion,Comuna,Localidad,Kobo,...,Variación ganado ovino (%),Variación ganado bovino(%),Unnamed: 100,Fecha,Descripción,Archivos adjuntos,Unnamed: 104,Fecha.1,Descripción.1,Archivos adjuntos.1
0,Eduvigis,Mancilla,4.536.803-3,Eduvigis Mancilla,81.0,9 57145097\n9 86036385,"42° 32' 50.3999"" S 73° 42' 58.7002"" W",Castro,Lingue,True,...,"0,0%",NaN,Labores extras,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bernardita,Ignao,9.171.733-6,Bernardita Ignao,63.0,9 82203632,"42° 33' 29.52"" S 73° 42' 10.944"" W",Castro,La Estancia,True,...,"-28,6%",NaN,NaN,mayo 2025,Entrega de metarizium,NaN,NaN,NaN,NaN,NaN
2,Orieta,Mansilla,13.825.891-2,Orieta Mansilla,45.0,9 48642683,"42° 33' 48.8002"" S 73° 42' 53.3002"" W",Castro,La Estancia,True,...,"-100,0%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Humberto,Calisto,9.738.608-0,Humberto Calisto,61.0,9 78058163,"42° 32' 43.8"" S 73° 43' 5.8001"" W",Castro,Lingue,True,...,NaN,"-50,0%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hector,Soto,8.635.176-5,Hector Soto,65.0,9 71364145,"42° 32' 37.914"" S 73° 42' 10.656"" W",Castro,Ducán-La Estancia,True,...,"-60,0%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Columns: 108 entries, Nombre to Archivos adjuntos.1
dtypes: float64(15), object(93)
memory usage: 149.5+ KB


In [12]:
import re

def dms_a_decimal(coord):
    try:
        patron = r'(\d+)°\s*(\d+)\'\s*([\d\.]+)"\s*([NS])\s*(\d+)°\s*(\d+)\'\s*([\d\.]+)"\s*([EW])'
        m = re.search(patron, coord)

        if not m:
            return None, None

        lat_g, lat_m, lat_s, lat_dir, lon_g, lon_m, lon_s, lon_dir = m.groups()

        lat = float(lat_g) + float(lat_m)/60 + float(lat_s)/3600
        lon = float(lon_g) + float(lon_m)/60 + float(lon_s)/3600

        if lat_dir == 'S':
            lat = -lat
        if lon_dir == 'W':
            lon = -lon

        return lat, lon
    except:
        return None, None


In [16]:
df[["lat", "lon"]] = df["Ubicacion"].astype(str).apply(
    lambda x: pd.Series(dms_a_decimal(x))
)

df[["Ubicacion", "lat", "lon"]].head(1000)

import numpy as np

df = df.replace({np.nan: None})



aqui transformamos a geojson para graficar

In [1]:
import json

features = []

for _, row in df.iterrows():
    if pd.notna(row["lat"]) and pd.notna(row["lon"]):

        propiedades = row.drop(["lat", "lon"]).to_dict()

        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row["lon"], row["lat"]]
            },
            "properties": propiedades
        }

        features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

with open("puntos.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("GeoJSON generado con", len(features), "puntos")


NameError: name 'df' is not defined